<a href="https://colab.research.google.com/github/touseefyounas/PV_Analysis/blob/main/Financing_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
from google.colab import data_table
import requests
import pandas as pd
import plotly.express as px
import folium
import numpy as np
from folium.raster_layers import ImageOverlay
import plotly.graph_objects as go

In [24]:
url="https://developer.nrel.gov/api/pvwatts/v8"
api_key="Rrm78PdXJt41jDkniLFvy5M2NyFY90cxQd2xGopJ"
latitude=28.5384
longitude=-81.3789
params={'format':'json',
        'api_key':api_key,
        'system_capacity':10,
        'module_type':0,
        'losses':14,
        'array_type':1,
        'tilt':30,
        'azimuth':180,
        'lat':latitude,
        'lon':longitude,
        'albedo':0.3,
        'inv_eff':94}

response=requests.get(f"{url}.json",params=params)

if response.status_code==200:
  pv_data=response.json()

  #ac_monthly=pv_data['outputs']['ac_monthly']
  output_data=pv_data['outputs']
else:
  print(f"Error: {response.status_code} - {response.text}")

Months=['January','February','March','April','May','June','July','August','September','October','November','December']

#ac_pd=pd.DataFrame(table_data)
output_df=pd.DataFrame(output_data)
output_df.insert(0,'Month',Months)
output_df.head(12)

,Month,ac_monthly,poa_monthly,solrad_monthly,dc_monthly,ac_annual,solrad_annual,capacity_factor
0,January,1242.089162,162.888378,5.254464,1329.252449,15220.591388,5.662066,17.375104
1,February,1121.065586,148.239338,5.294262,1199.770034,15220.591388,5.662066,17.375104
2,March,1449.822502,194.065990,6.260193,1549.830575,15220.591388,5.662066,17.375104
3,April,1411.559471,192.550877,6.418363,1510.543458,15220.591388,5.662066,17.375104
4,May,1415.387101,195.265213,6.298878,1515.253391,15220.591388,5.662066,17.375104
5,June,1181.094225,167.335103,5.577837,1266.641177,15220.591388,5.662066,17.375104
6,July,1217.956547,171.613974,5.535935,1306.228647,15220.591388,5.662066,17.375104
7,August,1270.666189,178.926277,5.771815,1361.991528,15220.591388,5.662066,17.375104
8,September,1220.995086,168.206972,5.606899,1307.811318,15220.591388,5.662066,17.375104
9,October,1369.440419,183.434046,5.917227,1464.431076,15220.591388,5.662066,17.375104


In [49]:
total_cost=20000
down_payment=2000
interest_rate=5
term_years=20
incentives=5000
utility_growth_rate=4



def calculate_solar_financing(total_cost, down_payment, interest_rate, term_years, incentives):
    principal = total_cost - down_payment
    monthly_interest = interest_rate / 12 / 100
    total_payments = term_years * 12

    monthly_payment = principal * (monthly_interest * ((1 + monthly_interest) ** total_payments)) / (((1 + monthly_interest) ** total_payments) - 1)
    total_paid = monthly_payment * total_payments
    total_paid_after_incentives = total_paid - incentives
   #total_savings = (monthly_savings * 12 * term_years) - total_paid_after_incentives

    return {
        "monthly_payment": monthly_payment,
        "total_paid": total_paid,
        "total_paid_after_incentives": total_paid_after_incentives
        #"total_savings": total_savings
    }

final=calculate_solar_financing(total_cost,down_payment,interest_rate,term_years,incentives)
monthly_payment=final['monthly_payment']

118.79203305899857

In [58]:
#Calcualte increase in the utility rate for 2023

# Monthly average utility rates in cents per kWh
rates={'2021':13.58,
       '2022':14.92,
       '2023':15.97}

# Calculate the monthly increase rate
increase_rates = {}
years = list(rates.keys())
increase_rates=[]


for i in range(len(years) - 1):
    old_rate = rates[years[i]]
    new_rate = rates[years[i + 1]]
    increase_rate = ((new_rate - old_rate) / old_rate) * 100
    increase_rates.append(increase_rate)

# Print the increase rates
avg_rate_increase=sum(increase_rates)/len(increase_rates)

#Calculating average increase in monthly utility rate
average_rates = []
average_gen=output_df['ac_monthly'].mean()

#making a list for the rate for 25 years
years_gen = np.arange(1,26)
future_years = years_gen.tolist()

for i in range(len(future_years)):
  average_rate = (1.34*(i+1))+12.24
  average_bill=average_gen*(average_rate/100)
  average_rates.append(average_bill)



rates_df=pd.DataFrame(average_rates)

# Add a new column to rates_df
rates_df['Monthly Payment'] = [monthly_payment if i < 20 else 0 for i in range(len(rates_df))]
rates_df.rename(columns={0:'Average Monthly Bill'},inplace=True)

rates_df.head(5)

,Average Monthly Bill,Monthly Payment
0,172.246359,118.792033
1,189.242686,118.792033
2,206.239013,118.792033
3,223.235340,118.792033
4,240.231667,118.792033


In [57]:
fig=go.Figure()

fig.add_trace(go.Scatter(
    x=rates_df.index, y=rates_df['Average Monthly Bill'],
    ))
fig.add_trace(go.Scatter(
    x=rates_df.index, y=rates_df['Monthly Payment'],
    ))
#fig = px.line(rates_df, x=rates_df.index, y='Average Monthly Bill',hover_data='Average Monthly Bill',labels={'Average Monthly Bill':'Monthly Bill $'})
fig.update_layout(paper_bgcolor='#18191A',plot_bgcolor='#242526',font_color='#FFFFFF',coloraxis_showscale=False,
                  yaxis=dict(title='Value',showgrid=False),xaxis=dict(title=None,showgrid=False), showlegend=False,
                  margin=dict(l=20, r=20, b=10, t=20, pad=1))
fig.update_traces(fillcolor='rgba(245,161,39,0.3)')
fig.update_traces(line=dict(color='yellow'))


fig.show()